<a href="https://colab.research.google.com/github/JoseGabriel-ITD/Curso-IA-Python/blob/main/Practica_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

import pandas as pd

In [71]:
prestamos = pd.read_csv('prestamos.csv')
prestamos.head()

,funded_amnt,loan_term_year,int_rate,grade_code,purpose_code,addr_state_code,home_ownership_code,annual_inc,dti,revol_util,pub_rec_bankruptcies,repaid,total_pymnt
0,2400,3,15.96,2,11,2,4,12252,8.72,98.5,0,1,3005.666844
1,10000,3,13.49,2,9,0,4,49200,20.00,21.0,0,1,12231.890000
2,3000,3,18.64,4,0,0,4,48000,5.35,87.5,0,1,3939.135294
3,5600,5,21.28,5,11,0,3,40000,5.55,32.6,0,0,647.500000
4,5375,5,12.69,1,9,5,4,15000,18.08,36.5,0,0,1484.590000


In [72]:
# columnas necesarias para entrenar el modelo de clasificacion
model_columns = ['funded_amnt', "int_rate", "grade_code", 'purpose_code', 'addr_state_code',
                 'home_ownership_code', 'annual_inc', 'dti', 'revol_util',
                 'pub_rec_bankruptcies']

In [73]:
X = prestamos[model_columns]  # Selección de características
y = prestamos["repaid"]  # Variable objetivo

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [74]:
X_train.head()

,funded_amnt,int_rate,grade_code,purpose_code,addr_state_code,home_ownership_code,annual_inc,dti,revol_util,pub_rec_bankruptcies
17433,4400,11.83,1,6,0,0,48000,3.52,58.3,0
7540,8000,12.99,2,2,0,4,99000,23.93,93.4,0
4913,10000,5.42,0,4,0,0,96000,1.69,16.8,0
3679,19850,12.69,1,2,3,0,67000,18.84,24.5,0
17388,24250,16.70,4,2,4,4,126000,17.14,97.9,0


In [75]:
rfc1 = RandomForestClassifier(random_state=23)

# Fit estimator and display score
rfc1 = rfc1.fit(X_train, y_train)

pred = rfc1.predict(X_test)
print(f'Precisión: {rfc1.score(X_test, y_test)}')

print("Reporte de métricas del clasificador: \n", classification_report(y_test, pred))

print('Matriz Confusion:\n', confusion_matrix(y_test, pred))

Precisión: 0.8441737820190859
Reporte de métricas del clasificador: 
               precision    recall  f1-score   support

           0       0.36      0.02      0.04      1220
           1       0.85      0.99      0.92      6744

    accuracy                           0.84      7964
   macro avg       0.60      0.51      0.48      7964
weighted avg       0.77      0.84      0.78      7964

Matriz Confusion:
 [[  27 1193]
 [  48 6696]]


### **📊 Análisis de los Resultados de Random Forest (sin balanceo ni `max_depth`)**  

El modelo tiene una **precisión global del 84.4%**, pero está claramente **sesgado** hacia la clase mayoritaria (`1`, créditos pagados).

---

## **📌 1. Matriz de Confusión**  
| | **Predicho: No Pagado (0)** | **Predicho: Pagado (1)** |
|----|-------------------|------------------|
| **Real: No Pagado (0)** | **27** (TP) ✅ | **1193** (FN) ❌ |
| **Real: Pagado (1)** | **48** (FP) ❌ | **6696** (TN) ✅ |

👉 **¿Qué significa esto?**  
- **El modelo casi nunca predice "No Pagado" (`0`).** Solo detectó **27 de 1220 creditos que no se pagaran** (¡muy bajo!).  
- **La mayoría de los créditos que no se pagaran fueron mal clasificados como pagados (`1`)**, lo que es un gran problema si queremos detectar riesgos.  
- **Detecta bien los créditos pagados (`1`)**, con solo **48 errores en 6744 casos**.  

---

## **📌 2. Análisis del Reporte de Clasificación**  

| Clase | **Precisión** | **Recall** | **F1-score** | **Support** |
|-------|-------------|-----------|------------|----------|
| **0 (No Pagado)** | **0.36** (Muy bajo) | **0.02** (¡Extremadamente bajo!) | **0.04** | 1220 |
| **1 (Pagado)** | **0.85** (Buena) | **0.99** (Excelente) | **0.92** | 6744 |

👉 **¿Qué significa esto?**  
- **El recall de la clase 0 (`No Pagado`) es solo 2%** → **El modelo ignora casi todos los casos de impago**.  
- **La precisión de la clase 0 (`No Pagado`) es 36%** → Cuando el modelo dice que un crédito no será pagado, solo acierta el 36% de las veces.  
- **La clase 1 (`Pagado`) tiene métricas muy altas**, pero **no es suficiente porque el modelo no equilibra ambas clases**.  

---

## **📌 3. ¿Por qué pasa esto? (Problema de Desbalance de Clases)**  
- La cantidad de créditos pagados (`1`) es mucho mayor (6744) que los impagos (`0`, solo 1220).  
- **Random Forest tiende a favorecer la clase mayoritaria** si no se balancean las clases.  
- **No especificaste `max_depth`**, por lo que los árboles crecieron libremente.  

---

## **📌 4. ¿Cómo mejorar el modelo?**  
✅ **1. Balancear las clases** con `class_weight='balanced'`:  
Esto ajusta el peso de cada clase para que el modelo le dé más importancia a los impagos (`0`).  

  
✅ **2. Ajustar `max_depth`** para evitar que el modelo crezca sin control (prueba con `5-20`).  


## **Escalamiento de variables**

Random Forest utiliza árboles de decisión, y estos no son sensibles a la escala de las variables porque:  
✔️ Dividen los datos basándose en condiciones lógicas (>=, <), no en distancias.  
✔️ No dependen de cálculos basados en magnitudes como lo hacen los modelos lineales.  

### **Clasificador de bosque aleatorio con class_weight='balanced**

In [78]:
rfc2 = RandomForestClassifier( class_weight='balanced', random_state=23)

# Entrenamiento
rfc2 = rfc2.fit(X_train, y_train)

pred = rfc2.predict(X_test)
print(f'Precisión: {rfc2.score(X_test, y_test)}')

print("Reporte de métricas del clasificador: \n", classification_report(y_test, pred))

print(f'Confusion Matrix:\n{confusion_matrix(y_test, pred)}')

Precisión: 0.8451783023606229
Reporte de métricas del clasificador: 
               precision    recall  f1-score   support

           0       0.33      0.01      0.02      1220
           1       0.85      1.00      0.92      6744

    accuracy                           0.85      7964
   macro avg       0.59      0.50      0.47      7964
weighted avg       0.77      0.85      0.78      7964

Confusion Matrix:
[[  13 1207]
 [  26 6718]]


### **📊 Análisis de los Nuevos Resultados con `class_weight='balanced'`**  

Aunque ajustaste el balance de clases, **el problema persiste**: el modelo sigue favoreciendo **demasiado** a la clase mayoritaria (`1`, créditos pagados).  

---

## **📌 1. Matriz de Confusión**  
| | **Predicho: No Pagado (0)** | **Predicho: Pagado (1)** |
|----|-------------------|------------------|
| **Real: No Pagado (0)** | **13** (TP) ✅ | **1207** (FN) ❌ |
| **Real: Pagado (1)** | **26** (FP) ❌ | **6718** (TN) ✅ |

🔹 **Interpretación:**  
- **Solo detectó 13 de los 1220 impagos**, lo que es **peor** que antes (antes detectaba 27).  
- **Sigue clasificando casi todos los impagos como pagados (1207 errores)**.  
- **Apenas mejoró en la detección de impagos, a pesar de balancear pesos**.  

---

## **📌 2. Análisis del Reporte de Clasificación**  

| Clase | **Precisión** | **Recall** | **F1-score** | **Support** |
|-------|-------------|-----------|------------|----------|
| **0 (No Pagado)** | **0.33** | **0.01** (!) | **0.02** | 1220 |
| **1 (Pagado)** | **0.85** | **1.00** | **0.92** | 6744 |

🔹 **Problema clave:**  
- **El recall de la clase 0 (`No Pagado`) bajó de 2% a 1%** → sigue ignorando los impagos.  
- **El modelo sigue demasiado sesgado a la clase mayoritaria (`1`)**.  
- **Precisión global de 84.5% es engañosa** porque el modelo **no predice correctamente los impagos**.  

---

## **📌 3. ¿Por qué sigue sin mejorar la detección de impagos?**  
✅ **Aún hay un fuerte desbalance de clases en los datos**:  
   - Aunque `class_weight='balanced'` ajusta los pesos internamente, la diferencia en cantidad de muestras sigue afectando al modelo.  

✅ **`max_depth=None` permite que los árboles crezcan demasiado**  
   - Esto hace que el modelo se adapte demasiado a los datos mayoritarios.  

## **📌 4. ¿Cómo mejorarlo? (Soluciones recomendadas)**  

✅ **1. Ajustar `max_depth` para reducir sobreajuste**  
Probar con diferentes parametros de max_depth. Por ejemplo: `max_depth=10` o `max_depth=15`:  

✅ **2. Aumentar `min_samples_leaf` y `min_samples_split` para mejorar generalización**  


### **Max_depth**
El parámetro max_depth en Random Forest controla la profundidad máxima de cada árbol en el bosque.

**¿Para qué se usa?**  
- Controla la complejidad del modelo.
- Evita el sobreajuste (cuando los árboles son demasiado profundos y memorizan los datos en lugar de generalizar).
- Afecta el rendimiento: árboles más profundos pueden capturar más patrones, pero también ser más costosos computacionalmente.

**¿Cómo elegir el mejor max_depth?**  
Depende de los datos, pero aquí hay algunas reglas generales:

- Si max_depth es muy pequeño → El modelo será demasiado simple y puede tener bajo rendimiento (subajuste).  
- Si max_depth es muy grande → El modelo puede memorizar los datos de entrenamiento (sobreajuste) y no generalizar bien.
- Si max_depth no se especifica → Los árboles crecen hasta que todas las hojas contienen menos de min_samples_split ejemplos.  
  
**En la práctica:**
- Probar diferentes valores.
- Un valor típico está entre 3 y 20 (depende del dataset).

In [82]:
rfc3 = RandomForestClassifier( max_depth=10, class_weight='balanced', random_state=23)

# Entrenamiento
rfc3 = rfc3.fit(X_train, y_train)

pred = rfc3.predict(X_test)
print(f'Precisión: {rfc3.score(X_test, y_test)}')

print("Reporte de métricas del clasificador: \n", classification_report(y_test, pred))

print(f'Confusion Matrix:\n{confusion_matrix(y_test, pred)}')

Precisión: 0.7424660974384731
Reporte de métricas del clasificador: 
               precision    recall  f1-score   support

           0       0.25      0.35      0.29      1220
           1       0.87      0.81      0.84      6744

    accuracy                           0.74      7964
   macro avg       0.56      0.58      0.57      7964
weighted avg       0.78      0.74      0.76      7964

Confusion Matrix:
[[ 425  795]
 [1256 5488]]


### **📊 Análisis de los Nuevos Resultados con `max_depth=10`**  

La precisión global bajó un poco (**74% vs. 84% antes**), pero ahora el modelo **detecta más Creditos que no se pagarán**, lo cual es positivo.  

---

## **📌 1. Matriz de Confusión**  
| | **Predicho: No Pagado (0)** | **Predicho: Pagado (1)** |
|----|-------------------|------------------|
| **Real: No Pagado (0)** | **425** (TP) ✅ | **795** (FN) ❌ |
| **Real: Pagado (1)** | **1256** (FP) ❌ | **5488** (TN) ✅ |  

  **Comparación con la matriz anterior (`max_depth=None`)**  

| | **Antes (`max_depth=None`)** | **Ahora (`max_depth=10`)** |
|----|-----------------|-----------------|
| TP (No Pagado detectado) | **13** | **425** (⬆ Mejoró mucho) |
| FN (No Pagado mal clasificado) | **1207** | **795** (⬇ Mejoró) |
| FP (Pagado mal clasificado) | **26** | **1256** (⬆ Empeoró) |
| TN (Pagado detectado) | **6718** | **5488** (⬇ Peor, pero más equilibrado) |

📌 **Conclusión:**  
- **Ahora detecta muchos más usuarios morosos (`425` en vez de `13`)**, lo cual es una gran mejora.  
- **Todavía hay muchos falsos negativos (`795` morosos mal clasificados)**, pero menos que antes.  
- **Ahora comete más falsos positivos (`1256`), es decir, clasifica erróneamente algunos créditos pagados como no pagados.**  
- **Es un mejor modelo para detectar riesgos de No pagos**, aunque bajó un poco la precisión global.  

---

## **📌 2. Análisis del Reporte de Clasificación**  

| Clase | **Precisión** | **Recall** | **F1-score** | **Support** |
|-------|-------------|-----------|------------|----------|
| **0 (No Pagado)** | **0.25** | **0.35** | **0.29** | 1220 |
| **1 (Pagado)** | **0.87** | **0.81** | **0.84** | 6744 |  


**Mejoras y puntos clave:**   
- **El recall de la clase `0` (No Pagado) subió de `0.01` a `0.35`** → Ahora detecta **35% de los que no van a pagr** (antes solo detectaba **1%**). 🚀  
- **La precisión de `0` es baja (25%), lo que significa que aún hay muchos falsos positivos** (detecta como no pagados algunos créditos pagados).  
- **El recall de `1` bajó de `1.00` a `0.81`**, lo que significa que perdió algo de capacidad para detectar créditos pagados.  
- **El modelo es más equilibrado, pero hay margen de mejora.**  

---

## **📌 3. ¿Cómo seguir mejorando el modelo?**  

✅ **1. Ajustar el umbral de decisión (`threshold`)**  
Ahora mismo, el umbral es `0.5`, pero si lo bajas a `0.4` o `0.3`, puede mejorar la detección de impagos.  

```python
y_prob = rf_balanced.predict_proba(X_test)[:, 1]
y_pred_new = (y_prob > 0.4).astype(int)  # Cambia el umbral a 0.4
```

✅ **2. Probar `max_depth=15` y `min_samples_leaf=5`**  
Esto puede mejorar el balance entre precisión y recall.  

✅ **3. Aplicar SMOTE para generar más ejemplos de impagos (`0`)**  
El desbalance aún afecta al modelo. **SMOTE puede ayudar a crear más datos de la clase `0`** para que el modelo los aprenda mejor.  


In [83]:
rfc4 = RandomForestClassifier( max_depth=7, class_weight='balanced', random_state=23)

# Entrenamiento
rfc4 = rfc4.fit(X_train, y_train)

pred = rfc4.predict(X_test)
print(f'Precisión: {rfc4.score(X_test, y_test)}')

print("Reporte de métricas del clasificador: \n", classification_report(y_test, pred))

print(f'Confusion Matrix:\n{confusion_matrix(y_test, pred)}')

Precisión: 0.6530637870416875
Reporte de métricas del clasificador: 
               precision    recall  f1-score   support

           0       0.23      0.54      0.32      1220
           1       0.89      0.67      0.77      6744

    accuracy                           0.65      7964
   macro avg       0.56      0.61      0.55      7964
weighted avg       0.79      0.65      0.70      7964

Confusion Matrix:
[[ 661  559]
 [2204 4540]]


## **📌 1. Matriz de Confusión**  
| | **Predicho: No Pagado (0)** | **Predicho: Pagado (1)** |
|----|-------------------|------------------|
| **Real: No Pagado (0)** | **661** (TP) ✅ | **559** (FN) ❌ |
| **Real: Pagado (1)** | **2204** (FP) ❌ | **4540** (TN) ✅ |

- Verdaderos positivos (TP): 661. El modelo clasificó correctamente 661 préstamos como incumplidos.  
- Falsos negativos (FN): 559.  El modelo predijo incorrectamente 559 préstamos como "pagado" (cuando en realidad no fueron pagados).
- Falsos positivos (FP): 2204. El modelo predijo incorrectamente 2204 préstamos como "no pagado" (cuando en realidad fueron pagados)   
- Verdaderos negativos (TN): 4540. El modelo clasificó correctamente 4540 préstamos como pagados.  


